In [5]:
import numpy as np
import pandas as pd

In [6]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

## Загрузка данных

In [7]:
train_df = pd.read_csv("train.csv")
train_df = train_df.dropna()
train_df.head()

,ID,url,title,label
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, цель которого сделать из республики не просителя, а донора: Надо избегать долгого нахождения н�",0
1,1,www.kp.by,Эта песня стала известна многим телезрителям благодаря сериалу Диверсант-2,0
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть онлайн!,0
3,3,colorbox.spb.ru,Не Беси Меня Картинки,0
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алексинской «Звезды» и сборной Китая | Т...,0


In [8]:
test_df = pd.read_csv("test.csv")
test_df = test_df.fillna('')  
test_df.head()

,ID,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу в разных зданиях – Фото – Коммерсантъ
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких случаях производится, каковы порядок и правила процедуры?"
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и национальностей онлайн"
4,135313,2gis.ru,67


In [10]:
class_counts = train_df['label'].value_counts()
print("Class counts:\n", class_counts)

Class counts:
 label
0    118593
1     16715
Name: count, dtype: int64


## Предобработка данных

In [11]:
bad_words = ['porn', 'порн', 'sex', 'секс', 'xxx', 'fap', 'fuck', 'milf', 'dick', 'gay', 'boob']

ignore_words = ['transex', 'трансекс']

smth_before_porn = r'.*[а-яА-Я]порн.*'

In [12]:
import re

def preprocess_url(url):

    # Разбиваем URL на токены по разделителю '.'
    tokens = url.split('.')
    
    # Удаляем первый токен, если это 'www'
    if tokens[0] == 'www':
        tokens = tokens[1:]
    
    # Проверяем, является ли какой-либо токен IP-адресом
    ip_pattern = r'^\d+\.\d+\.\d+\.\d+'
    for token in tokens:
        if re.match(ip_pattern, token):
            return ""  # Удаляем весь URL, если есть IP-адрес
    
    # Удаляем URL, если есть токен 'xn--'
    if 'xn--' in tokens:
        return ""
    
    # Удаляем последний токен, если он не равен 'porn'
    if tokens[-1] != 'porn':
        tokens = tokens[:-1]

    for i, token in enumerate(tokens):
        if ignore_words[0] not in token and ignore_words[1] not in token and not re.match(smth_before_porn, token):
            for bw in bad_words:
                if bw in token:
                    tokens[i] = bw
               
    cleaned_tokens = []
    for token in tokens:
        # Удаляем специальные символы
        cleaned_token = re.sub(r'[^a-zA-Z]', '', token)
        if cleaned_token:
            cleaned_tokens.append(cleaned_token)
            
    processed_url = ' '.join(cleaned_tokens)
    
    return processed_url

train_df['url'] = train_df['url'].apply(preprocess_url)
test_df['url'] = test_df['url'].apply(preprocess_url)

In [13]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

en_stemmer = PorterStemmer()

# Стемминг
def morph_process_url(text):
    stemmed_words = []
    words = word_tokenize(text)
    for word in words:
        stemmed_word = en_stemmer.stem(word)
        stemmed_words.append(stemmed_word)
    return ' '.join(stemmed_words)

train_df['url'] = train_df['url'].apply(morph_process_url) 
test_df['url'] = test_df['url'].apply(morph_process_url)

In [14]:
import string
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words_eng = set(stopwords.words('english'))
stop_words_rus = set(stopwords.words('russian'))

def check_string(s):
    russian_regex = r'^[а-яА-Я]+$'
    latin_regex = r'^[a-zA-Z]+$'
    
    if re.match(russian_regex, s):
        return True
    elif re.match(latin_regex, s):
        return True
    else:
        return False

def preprocess_text(text):
    # Преобразование текста в нижний регистр
    text = text.lower()

    # Удаление лишних пробелов
    text = ' '.join(text.split())

    # Замена знаков препинания на пробелы
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))

    # Удаление спецсимволов
    text = re.sub(r'[^\w\s]', ' ', text)

    # Замена всех "ё" на "е"
    text = text.replace('ё', 'е')

    for token in text.split():
        if ignore_words[0] not in token and ignore_words[1] not in token and not re.match(smth_before_porn, token):
            for bw in bad_words:
                if bw in token:
                    text = text.replace(token, bw)

    # Удаление цифр и коротких/длинных слов
    text = ' '.join(word for word in text.split() if 3 <= len(word) <= 15 and not any(char.isdigit() for char in word))
    
    # Удаление стоп-слов на русском и английском языках
    text = ' '.join(word for word in text.split() if word not in stop_words_eng and word not in stop_words_rus)
    
    # Замена на пустые строки, если строка не соответствует ни русскому, ни английскому языку
    text = ' '.join('' if not check_string(word) else word for word in text.split())
    
    return text

train_df['title'] = train_df['title'].apply(preprocess_text)  
test_df['title'] = test_df['title'].apply(preprocess_text)    

[nltk_data] Downloading package stopwords to /home/roman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
from nltk.stem import PorterStemmer
import pymorphy3
from nltk.tokenize import word_tokenize

en_stemmer = PorterStemmer()
ru_stemmer = pymorphy3.MorphAnalyzer()

# Стемминг и лемматизация
def morph_process(text):
    stemmed_words = []
    words = word_tokenize(text)
    for word in words:
        if word.isascii():
            stemmed_word = en_stemmer.stem(word)
        else:
            stemmed_word = ru_stemmer.parse(word)[0].normal_form
        stemmed_words.append(stemmed_word)
    return ' '.join(stemmed_words)

train_df['title'] = train_df['title'].apply(morph_process)  
test_df['title'] = test_df['title'].apply(morph_process)   

In [16]:
train_df['combined_text'] = train_df['url'] + ' ' + train_df['title']
test_df['combined_text'] = test_df['url'] + ' ' + test_df['title']

In [17]:
train_df.head()

,ID,url,title,label,combined_text
0,0,m kp,экс министр экономика молдова глава мидэя цель который сделать республика проситель донор избегать долгий нахождение,0,m kp экс министр экономика молдова глава мидэя цель который сделать республика проситель донор избегать долгий нахождение
1,1,kp,этот песня стать известный многий телезритель благодаря сериал диверсант,0,kp этот песня стать известный многий телезритель благодаря сериал диверсант
2,2,fanseri,банша сезон серия бремя красота смотреть онлайн,0,fanseri банша сезон серия бремя красота смотреть онлайн
3,3,colorbox spb,бесить картинка,0,colorbox spb бесить картинка
4,4,tulasport,новомосковск сыграть следж хоккеист алексинский звезда сборная китай,0,tulasport новомосковск сыграть следж хоккеист алексинский звезда сборная китай


In [18]:
test_df.head()

,ID,url,title,combined_text
0,135309,kommers,шестой кассационный суд самара начать работа разный здание фото коммерсантъ,kommers шестой кассационный суд самара начать работа разный здание фото коммерсантъ
1,135310,urexpert,такой индексация алименты кто какой случай производиться каковой порядок правило процедура,urexpert такой индексация алименты кто какой случай производиться каковой порядок правило процедура
2,135311,imperimeha,женщина империя мех part,imperimeha женщина империя мех part
3,135312,porn,небритый волосатый киска порн страна национальность онлайн,porn небритый волосатый киска порн страна национальность онлайн
4,135313,gi,,gi


## Применение модели

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

X = train_df['combined_text']
y = train_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(solver='liblinear'))
])

parameters = {
    'tfidf__max_df': (0.5, 0.75, 1.0),
    'tfidf__ngram_range': [(1, 2), (1, 3)],
    'clf__C': (1, 100),
}

grid_search = RandomizedSearchCV(pipeline, parameters, cv=5, scoring='f1', n_jobs=-1, n_iter=6)
grid_search.fit(X_train, y_train)

print("Наилучшие параметры:")
print(grid_search.best_params_)

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

Наилучшие параметры:
{'tfidf__ngram_range': (1, 2), 'tfidf__max_df': 0.5, 'clf__C': 100}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23662
           1       0.98      0.98      0.98      3400

    accuracy                           1.00     27062
   macro avg       0.99      0.99      0.99     27062
weighted avg       1.00      1.00      1.00     27062



In [20]:
# Используем выбранные параметры
tfidf_params = {
    'max_df': 0.5,
    'ngram_range': (1, 2)
}

clf_params = {
    'C': 100
}

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(**tfidf_params)),
    ('clf', LogisticRegression(solver='liblinear', **clf_params))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred, digits=5))


              precision    recall  f1-score   support

           0    0.99675   0.99776   0.99725     23662
           1    0.98430   0.97735   0.98081      3400

    accuracy                        0.99520     27062
   macro avg    0.99053   0.98756   0.98903     27062
weighted avg    0.99519   0.99520   0.99519     27062



## Формирование тестовых предсказаний

In [21]:
X_real_test = test_df['combined_text']
y_real_pred = pipeline.predict(X_real_test)

predictions_df = pd.DataFrame({'ID': test_df['ID'], 'label': y_real_pred})

predictions_df.to_csv('predictions.csv', index=False)